In [1]:
import os
os.getcwd()
os.chdir('..')

In [2]:
# Import Package
from utils import imageUtils
from utils import postProcessing
from utils import MultiDefectDetectionDataset
from utils import evaluation
from utils import visualization
import numpy as np
import os
from chainercv.links import FasterRCNNVGG16
from chainercv.visualizations import vis_bbox
from chainercv.utils import write_image
import chainer
import math
from chainercv import utils
%matplotlib inline
import matplotlib.pyplot as plt

#load Data
root = './data/3Types/Data3TypesYminXminYmaxXmax9'
dataset_test = MultiDefectDetectionDataset(data_dir=root, split='test')
dataset_human = MultiDefectDetectionDataset(data_dir=root, split='Stablehuman')
bbox_label_names = ('111', 'dot','100')
# DataSet Statistics
print('total number of training images: ', len(dataset_test))
print('total number of test images: ', len(dataset_test))
print('type of defects: ', bbox_label_names)

/Users/mingrenshen/anaconda3/envs/chainercv/lib/python2.7/site-packages/numpy/distutils/system_info.py:638: UserWarning: 
    Atlas (http://math-atlas.sourceforge.net/) libraries not found.
    Directories to search for the libraries can be specified in the
    numpy/distutils/site.cfg file (section [atlas]) or by setting
    the ATLAS environment variable.
  self.calc_info()
/Users/mingrenshen/anaconda3/envs/chainercv/lib/python2.7/site-packages/numpy/distutils/system_info.py:638: UserWarning: 
    Blas (http://www.netlib.org/blas/) libraries not found.
    Directories to search for the libraries can be specified in the
    numpy/distutils/site.cfg file (section [blas]) or by setting
    the BLAS environment variable.
  self.calc_info()
/Users/mingrenshen/anaconda3/envs/chainercv/lib/python2.7/site-packages/numpy/distutils/system_info.py:638: UserWarning: 
    Blas (http://www.netlib.org/blas/) sources not found.
    Directories to search for the sources can be specified in the
    nu

('total number of training images: ', 15)
('total number of test images: ', 15)
('type of defects: ', ('111', 'dot', '100'))


# Generate Basic BBox infomation and Crop Images

In [ ]:
# Faster R-CNN model

use_gpu = False
# Higher NMS
proposal_params = {'min_size': 8,'nms_thresh':0.4}
model = FasterRCNNVGG16(n_fg_class=3,
                        pretrained_model='modelResults/snapshot_model_244000_Shufulle.npz',
                        ratios=[ 0.5, 1, 1.5, 2, 2.5, 4,8,16], anchor_scales=[1, 4, 8, 16], min_size=1024,
                        max_size=1024, proposal_creator_params=proposal_params)

model.score_thresh = 0.25

if use_gpu:
    chainer.cuda.get_device_from_id(0).use()
    model.to_gpu()

In [ ]:
# Loop Thought Images in testing data set
for imgdata in dataset_human:
    # Figure Setting
    fig = plt.figure(figsize=(36,10))
    plt.axis('off')
    img, bbox, labels = imgdata
    fname = "img" + str(i)
    # Ground Truth
    ax1 = fig.add_subplot(1, 3, 1)
    visualization.vis_bbox(img, bbox, labels, label_names=bbox_label_names, ax=ax1)
    
    # Prediction of image
    # Predicted BBoxes
    ax2 = fig.add_subplot(1, 3, 2)
    bboxes, labels, scores = model.predict([img])
    # plot prediction
    visualization.vis_bbox(img, bboxes[0], labels[0], scores[0], label_names=bbox_label_names, ax=ax2)
    
    # Fitting of images
    ax3 = fig.add_subplot(1, 3, 3)
    img_ellipse_fitting_3kinds_Fig3(img, bboxes[0], labels[0],ax=ax3)
    
    
    fig.savefig(fname,dpi=300,bbox_inches='tight')
    i = i + 1

print("Done")